In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset

social = pd.read_csv("Aggregated_reddit_twitter.csv")
social = social.rename(columns={"date": "timestamp"})
social["timestamp"] = pd.to_datetime(social["timestamp"])

/tmp/ipykernel_294804/4215461173.py:5: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  social = pd.read_csv("Aggregated_reddit_twitter.csv")


In [2]:
social = social[social.content != "u/circular360"]
social = social[social.content != "There you go"]
social = social[
    social.content != "Why force the other player to stay around? Just give the winning player the option to continue the game with a perma passing AI controlling the surrendering player."]
# crypto = pd.read_csv("transformed_crypto.csv")
# crypto["timestamp"] = pd.to_datetime(crypto["timestamp"]).dt.tz_localize(None)
crypto = pd.read_csv("crypto_differences.csv")
crypto["timestamp"] = pd.to_datetime(crypto["timestamp"]).dt.tz_localize(None)

social = social.sort_values(by="timestamp")
# social.dtypes
sr = social.resample("T", on="timestamp").size()
freq = sr.reset_index()

freq = freq.rename(columns={0: "n_comments"})

In [3]:
freq["hour"] = freq["timestamp"].dt.hour
freq["day"] = freq["timestamp"].dt.dayofweek
freq["month"] = freq["timestamp"].dt.month
freq

,timestamp,n_comments,hour,day,month
0,2021-03-01 00:00:00,18,0,0,3
1,2021-03-01 00:01:00,8,0,0,3
2,2021-03-01 00:02:00,4,0,0,3
3,2021-03-01 00:03:00,11,0,0,3
4,2021-03-01 00:04:00,4,0,0,3
...,...,...,...,...,...
1132079,2023-04-26 03:59:00,0,3,2,4
1132080,2023-04-26 04:00:00,0,4,2,4
1132081,2023-04-26 04:01:00,0,4,2,4
1132082,2023-04-26 04:02:00,0,4,2,4


In [4]:
alp = pd.read_csv("df_alphavantage_news_with_finbert_sentiments.csv", low_memory=False)
alp["timestamp"] = pd.to_datetime(alp["time_published"])
# alp["timestamp"] = pd.to_datetime(alp["timestamp"])
alp = alp.drop("Unnamed: 0", axis=1)
alp = alp.drop("day", axis=1)

# alp = alp.fillna(0)
frqn = pd.merge(freq, alp, on="timestamp", how="left")
frqn = frqn.dropna(how="all", subset=["overall_sentiment_score", "BTC_relevance", "BTC_sentiment", "ETH_relevance",
                                      "ETH_sentiment",
                                      "finbert_title_sentiment", "finbert_summary_sentiment"
                                      ])
frqn = frqn.fillna(0)
# list(frqn["finbert_title_sentiment"])
frqn

,timestamp,n_comments,hour,day,month,title,url,time_published,authors,summary,...,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment,BTC_sentiment,ETH_sentiment,BTC_relevance,ETH_relevance,finbert_title_sentiment,finbert_summary_sentiment
527520,2022-03-02 08:00:00,0,8,2,3,"Business Highlights: Lobbyists leaving, rate h...",https://apnews.com/article/russia-ukraine-tech...,2022-03-02 08:00:00,[],"Business Highlights: Lobbyists leaving, rate h...",...,"[{'topic': 'Financial Markets', 'relevance_sco...",-0.174572,Somewhat-Bearish,"[{'ticker': 'CRYPTO:BTC', 'relevance_score': '...",-0.009523,0.000000,0.125701,0.000000,-0.459106,-0.466663
528960,2022-03-03 08:00:00,0,8,3,3,"Powell says rate hike is still coming, but not...",https://www.cnn.com/2022/03/02/economy/powell-...,2022-03-03 08:00:00,"['Anneken Tappe', 'CNN Business']",New York ( CNN Business ) The time of ultr...,...,"[{'topic': 'Blockchain', 'relevance_score': '0...",-0.016177,Neutral,"[{'ticker': 'FOREX:RUB', 'relevance_score': '0...",-0.178949,0.000000,0.073955,0.000000,-0.627116,-0.113677
528961,2022-03-03 08:00:00,0,8,3,3,Crypto 'superpower?' India's young investors p...,https://www.cnn.com/2022/03/03/investing/india...,2022-03-03 08:00:00,['Diksha Madhok'],Crypto 'superpower?' India's young investors p...,...,"[{'topic': 'Blockchain', 'relevance_score': '0...",-0.006920,Neutral,"[{'ticker': 'COIN', 'relevance_score': '0.0314...",0.019472,0.133847,0.156339,0.031462,-0.126144,-0.040588
530401,2022-03-04 08:00:00,0,8,4,3,Conflict With Russia And Inflation Give Invest...,https://www.investors.com/etfs-and-funds/stock...,2022-03-04 08:00:00,"[""Investor's Business Daily"", 'MARIE BEERENS']",Stock Market Suffers Fits From Conflict With R...,...,"[{'topic': 'Financial Markets', 'relevance_sco...",-0.260707,Somewhat-Bearish,"[{'ticker': 'AAPL', 'relevance_score': '0.0687...",-0.769721,0.000000,0.034430,0.000000,-0.521833,-0.935646
530402,2022-03-04 08:00:00,0,8,4,3,Stock market news today: Dow and S&P 500 updates,https://www.cnn.com/business/live-news/stock-m...,2022-03-04 08:00:00,['By CNN Business'],Live updates: Dow tumbles after Russian nuclea...,...,"[{'topic': 'Financial Markets', 'relevance_sco...",-0.460275,Bearish,"[{'ticker': 'MCO', 'relevance_score': '0.12478...",-0.415272,0.000000,0.041748,0.000000,-0.031041,-0.921851
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1132365,2023-04-25 19:00:00,0,19,1,4,Bitcoin's price is down 10% - New bear market ...,https://cointelegraph.com/news/bitcoin-s-price...,2023-04-25 19:00:00,[],"On this week's episode of The Market Report, C...",...,"[{'topic': 'Economy - Monetary', 'relevance_sc...",-0.079264,Neutral,"[{'ticker': 'GOOG', 'relevance_score': '0.1934...",-0.140288,-0.261726,0.375722,0.286640,-0.929247,0.000000
1132372,2023-04-25 19:07:00,0,19,1,4,2023 layoff tracker: The latest on which compa...,https://www.cnn.com/2023/04/25/business/layoff...,2023-04-25 19:07:00,['Jennifer Korn'],2023 layoff tracker: The latest on which compa...,...,"[{'topic': 'Technology', 'relevance_score': '0...",-0.033546,Neutral,"[{'ticker': 'BLK', 'relevance_score': '0.02624...",0.123520,0.000000,0.026241,0.000000,-0.955543,0.000000
1132500,2023-04-25 21:15:00,0,21,1,4,Bitcoin advocates rally at Texas State Capitol...,https://cointelegraph.com/news/bitcoin-advocat...,2023-04-25 21:15:00,[],"""When you target one industry the way they are...",...,"[{'topic': 'Life Sciences', 'relevance_score':...",0.004738,Neutral,"[{'ticker': 'MIGI', 'relevance_score': '0.0761...",-0.030772,0.000000,0.297661,0.000000,-0.730707,-0.768360
1132545,2023-04-25 22:00:00,0,22,1,4,Mercurity Fintech Holding Inc. Reports Full Fi...,https://www.globenewswire.com/news-release/202...,2023-04-25 22:00:00,['Mercurity Fintech Holding Inc.'],"NEW YORK, April 25, 2023 ( GLOBE NEWSWIRE ) --...",...,"[{'topic': 'Economy - Monetary', 'relevance_sc...",0.178590,Somewhat-Bullish,"[{'ticker': 'MFH', 'relevance_score': '0.42

In [7]:
mg = pd.merge(crypto, frqn, on="timestamp", how="inner")
shortmg = mg.loc[:]
shortmg.dropna()
print(shortmg.shape)
shortmg.describe()

(5042, 30)


,Unnamed: 0,XRP-PERP,SOL-PERP,ETH-PERP,DOGE-PERP,BTC-PERP,timestamp,n_comments,hour,day,month,overall_sentiment_score,BTC_sentiment,ETH_sentiment,BTC_relevance,ETH_relevance,finbert_title_sentiment,finbert_summary_sentiment
count,5042.000000,5042.000000,5042.000000,5042.000000,5042.000000,5042.000000,5042,5042.000000,5042.000000,5042.000000,5042.000000,5042.000000,5042.000000,5042.000000,5042.000000,5042.000000,5042.000000,5042.000000
mean,742030.064459,0.000703,0.000725,0.000120,0.000247,0.000434,2022-05-31 10:56:48.230860800,0.023998,13.317731,2.398453,5.486117,-0.008659,-0.018618,0.011107,0.249304,0.112311,-0.046986,0.015696
min,612269.000000,-1.604943,-1.607555,-5.859690,-2.765242,-1.404820,2022-03-02 08:00:00,0.000000,0.000000,0.000000,3.000000,-0.900288,-0.983580,-0.983527,0.000000,0.000000,-0.967714,-0.969429
25%,683128.250000,-0.000956,-0.001505,-0.001493,-0.000827,-0.001234,2022-04-20 13:00:15,0.000000,10.000000,1.000000,4.000000,-0.102619,-0.072256,0.000000,0.046795,0.000000,-0.274682,-0.054744
50%,740749.500000,0.000000,0.000000,0.000033,-0.000022,0.000000,2022-05-30 13:21:30,0.000000,13.000000,2.000000,5.000000,-0.001920,0.000000,0.000000,0.126806,0.012061,0.020006,0.014671
75%,799465.000000,0.000975,0.001474,0.001681,0.000738,0.001249,2022-07-10 08:39:00,0.000000,17.000000,4.000000,7.000000,0.089223,0.040335,0.019594,0.389835,0.133378,0.154556,0.190878
max,875221.000000,3.790067,3.448106,3.710992,1.737893,3.666007,2022-08-31 23:15:00,4.000000,23.000000,6.000000,8.000000,0.698813,0.968406,0.816919,0.996870,0.999718,0.940115,0.942435
std,71237.687316,0.072009,0.056235,0.117222,0.050578,0.067001,NaN,0.179320,5.002063,1.695327,1.615336,0.163062,0.179747,0.124918,0.276317,0.194894,0.489456,0.472229


In [8]:
class TimeSeriesDataset(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data

    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]

    def __len__(self):
        return len(self.X_data)


class MyTimeSeriesDataset(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data

    def __getitem__(self, index):
        return self.X_data.loc[index], self.y_data[index]

    def __len__(self):
        return len(self.X_data)

# data = MyTimeSeriesDataset(X_data=shortmg[['n_comments', 'XRP-PERP', 'ETH-PERP']], y_data=shortmg['BTC-PERP'])

In [109]:
# Load your dataset into a pandas DataFrame called 'data'
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Scale the features using MinMaxScaler
cs = ['BTC-PERP', 'XRP-PERP', 'ETH-PERP', 'SOL-PERP', 'DOGE-PERP', 'overall_sentiment_score', 'BTC_sentiment',
      'ETH_sentiment', 'BTC_relevance', 'ETH_relevance',
      'finbert_title_sentiment', 'finbert_summary_sentiment'
      ]
scaled_data = shortmg[cs + ["hour", "day", "month", "n_comments"]]
# scaled_data = shortmg[cs + ["hour", "day", "month"]]
# scaled_data = shortmg[['XRP-PERP', 'ETH-PERP', 'BTC-PERP', 'SOL-PERP', 'DOGE-PERP', 'n_comments']].to_numpy()

# scaled_data = abs(scaled_data)


# scaled_data = scaled_data.to_numpy()
#
# scaler = StandardScaler()
#
scaled_data = scaled_data.to_numpy()
# scaled_data = scaler.fit_transform(scaled_data)


# Create sequences and targets for the RNN model
seq_length = 30
X_data = []
y_data = []

future = 30

for i in range(len(scaled_data) - seq_length - future - 1):
    print(f"{i} / {len(scaled_data) - seq_length - future}")
    # if i > 100:
    #     break
    X_data.append(scaled_data[i:i + seq_length].flatten())
    # y_data.append(sum(crypto["timestamp"].between_time(
    #     shortmg["timestamp"][i + seq_length], shortmg["timestamp"][i + seq_length + future])
    # ))
    # print(crypto.dtypes, shortmg.dtypes)
    mask = (crypto["timestamp"] > shortmg["timestamp"][i + seq_length]) & (
            crypto["timestamp"] < shortmg["timestamp"][1 + i + seq_length + future])
    x = crypto[mask]
    y_data.append(sum(x["BTC-PERP"]))

# x_data = scaler.fit_transform(X_data)
# y_data = np.delete(y_data, [-1, -2, -3, -4], axis=1)
# y_data = np.delete(y_data, [-1, -2, -3, -4, -5, -6, -7, -8], axis=1)
# y_data = np.array(y_data).T[0].T
y_data = np.array(y_data)
# to make classifier
epsilon = 0.01


# epsilon = 0.0


def f(x):
    if x >= epsilon:
        return 1
    elif x < -epsilon:
        return 2
    else:
        return 0


# print(y_data)

y_data = list(map(f, y_data))

print(y_data[1:10])
X_data, y_data = np.array(X_data), np.array(y_data)

y_data = y_data.astype(np.int64)


0 / 4982
1 / 4982
2 / 4982
3 / 4982
4 / 4982
5 / 4982
6 / 4982
7 / 4982
8 / 4982
9 / 4982
10 / 4982
11 / 4982
12 / 4982
13 / 4982
14 / 4982
15 / 4982
16 / 4982
17 / 4982
18 / 4982
19 / 4982
20 / 4982
21 / 4982
22 / 4982
23 / 4982
24 / 4982
25 / 4982
26 / 4982
27 / 4982
28 / 4982
29 / 4982
30 / 4982
31 / 4982
32 / 4982
33 / 4982
34 / 4982
35 / 4982
36 / 4982
37 / 4982
38 / 4982
39 / 4982
40 / 4982
41 / 4982
42 / 4982
43 / 4982
44 / 4982
45 / 4982
46 / 4982
47 / 4982
48 / 4982
49 / 4982
50 / 4982
51 / 4982
52 / 4982
53 / 4982
54 / 4982
55 / 4982
56 / 4982
57 / 4982
58 / 4982
59 / 4982
60 / 4982
61 / 4982
62 / 4982
63 / 4982
64 / 4982
65 / 4982
66 / 4982
67 / 4982
68 / 4982
69 / 4982
70 / 4982
71 / 4982
72 / 4982
73 / 4982
74 / 4982
75 / 4982
76 / 4982
77 / 4982
78 / 4982
79 / 4982
80 / 4982
81 / 4982
82 / 4982
83 / 4982
84 / 4982
85 / 4982
86 / 4982
87 / 4982
88 / 4982
89 / 4982
90 / 4982
91 / 4982
92 / 4982
93 / 4982
94 / 4982
95 / 4982
96 / 4982
97 / 4982
98 / 4982
99 / 4982
100 / 4982

In [110]:
train_size = int(len(X_data) * 0.8)
X_train, X_test = X_data[:train_size], X_data[train_size:]
y_train, y_test = y_data[:train_size], y_data[train_size:]

In [111]:
from collections import Counter

Counter(y_train), Counter(y_test)

(Counter({2: 1723, 0: 385, 1: 1876}), Counter({1: 458, 0: 236, 2: 303}))

In [112]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, balanced_accuracy_score, accuracy_score, confusion_matrix

# regressor = RandomForestRegressor(random_state=0, max_depth=5, n_estimators=20)
# regressor = SVR()
regressor = LogisticRegression()
regressor.fit(X_train, y_train)
pred = regressor.predict(X_test)

balanced_accuracy_score(y_test, pred), accuracy_score(y_test, pred)

/home/dario/PycharmProjects/BAINSACryptocurrencies/venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.3618960926512309, 0.35606820461384153)

In [113]:
pred = regressor.predict(X_train)

print("TRAIN ACCURACY:  ", balanced_accuracy_score(pred, y_train), accuracy_score(pred, y_train))
print(confusion_matrix(y_train, pred))

TRAIN ACCURACY:   0.5634320496973856 0.5891064257028112
[[  59  175  151]
 [  19 1225  632]
 [  38  622 1063]]


In [114]:
# balanced_accuracy_score(y_test, [2] * len(y_test))

In [115]:
# import statsmodels.api as sm
# from scipy import stats
#
# X2 = sm.add_constant(X_train)
# y = y_train - 1
# est = sm.Logit(y, X2)
# est2 = est.fit()
# print(est2.summary())

In [116]:
pred = regressor.predict(X_test)
print(balanced_accuracy_score(pred, y_test), accuracy_score(pred, y_test))
print(confusion_matrix(y_test, pred))

0.38758606151390357 0.35606820461384153
[[ 12  44 180]
 [ 16  87 355]
 [  9  38 256]]


In [117]:
from torch.utils.data import DataLoader

batch_size = 64
train_dataset = TimeSeriesDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).float())
test_dataset = TimeSeriesDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).float())

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [118]:
# classifier
# y_data = y_data.flatten()
# y_data.shape

In [119]:
class DenseModel(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.lin = nn.Linear(input_size, 128)
        self.ff = nn.Sequential(
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, output_size)
        )

    def forward(self, x):
        # x = torch.flatten(x, 1)
        x = self.lin(x)
        x = self.ff(x)
        o = f.log_softmax(x, dim=1)
        return o

In [120]:
from torch import optim
import torch.nn.functional as f

input_size = X_data.shape[1]
# hidden_size = 256
# num_layers = 8
# output_size = y_data.shape[1]
output_size = 3
# output_size = 1

model = DenseModel(input_size, output_size)
optimizer = optim.Adam(model.parameters(), lr=0.001)
input_size, output_size

(480, 3)

In [121]:
import platform

num_epochs = 10
log = False
#
if "Linux" in platform.platform():
    model = torch.compile(model)
#
for epoch in range(num_epochs):
    model.train()
    correct = 0
    for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
        y_pred = model(X_batch)
        #
        optimizer.zero_grad()
        y_batch = y_batch.type(torch.LongTensor)
        loss = f.nll_loss(y_pred, y_batch)
        # pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        # correct += pred.eq(y_batch.view_as(pred)).sum().item()
        loss.backward()
        optimizer.step()
        # optimizer.step()

        if batch_idx % 20 == 0 and log:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(X_batch), len(train_loader.dataset), 100. * batch_idx / len(train_loader),
                loss))

    # print(f"Accuracy train: {100 * correct / len(train_loader.dataset)}")
    if True:
        model.eval()
        correct = 0
        test_loss = 0
        for X_batch, y_batch in test_loader:
            y_pred = model(X_batch)
            y_batch = y_batch.type(torch.LongTensor)

            loss = f.nll_loss(y_pred, y_batch)
            test_loss += loss

            pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(y_batch.view_as(pred)).sum().item()

        test_loss = test_loss / len(test_loader)

        print(
            f"Epoch: {epoch + 1}, Train Loss: {loss.item():.4f}, Test Loss: {test_loss:.4f}, accuracy: {100 * correct / len(test_loader.dataset)}")



[2023-05-31 16:15:32,010] torch._inductor.utils: [WARNING] using triton random, expect difference from eager
[2023-05-31 16:15:32,639] torch._inductor.utils: [WARNING] using triton random, expect difference from eager


Epoch: 1, Train Loss: 1.3409, Test Loss: 1.1125, accuracy: 45.937813440320966
Epoch: 2, Train Loss: 1.3907, Test Loss: 1.1080, accuracy: 45.937813440320966
Epoch: 3, Train Loss: 1.2580, Test Loss: 1.0611, accuracy: 46.138415245737214
Epoch: 4, Train Loss: 1.2658, Test Loss: 1.0662, accuracy: 34.302908726178536
Epoch: 5, Train Loss: 1.2064, Test Loss: 1.0736, accuracy: 35.10531594784353
Epoch: 6, Train Loss: 1.4814, Test Loss: 1.1052, accuracy: 40.92276830491475
Epoch: 7, Train Loss: 1.4172, Test Loss: 1.0863, accuracy: 42.126379137412236
Epoch: 8, Train Loss: 1.4938, Test Loss: 1.0837, accuracy: 43.02908726178536
Epoch: 9, Train Loss: 1.4114, Test Loss: 1.0751, accuracy: 39.81945837512538
Epoch: 10, Train Loss: 1.4057, Test Loss: 1.0886, accuracy: 37.311935807422266


In [124]:
ps = torch.exp(model(torch.Tensor(X_test)))
pred = ps.argmax(dim=1, keepdim=True)
print(balanced_accuracy_score(y_test, pred))
confusion_matrix(y_test, pred)

0.3991095468069492


array([[ 82,  33, 121],
       [115,  96, 247],
       [ 84,  25, 194]])

In [126]:
ps = torch.exp(model(torch.Tensor(X_train)))
pred = ps.argmax(dim=1, keepdim=True)
print(balanced_accuracy_score(y_train, pred))
confusion_matrix(y_train, pred)

0.4799206157332819


array([[ 102,   51,  232],
       [  49,  831,  996],
       [  61,  401, 1261]])